In [32]:
import time
import random

def double_point(P, a, p):
    if P is None:
        return None

    l = ((3 * P[0]**2 + a) * pow(2 * P[1], -1, p)) % p
    x = (l**2 - 2 * P[0]) % p
    y = ((l * (P[0] - x) - P[1]) % p)

    return (x,y)


def add_points(P, Q, a, p):

    if P is None:
        return Q
    if Q is None:
        return P

    if P == Q:
        return double_point(P, a, p)

    l = ((Q[1] - P[1]) * pow(Q[0] - P[0], -1, p)) % p
    x = (l ** 2 - P[0] - Q[0]) % p
    y = ((l * (P[0] - x) - P[1]) % p)

    return (x, y)


def binary_multiplication(P, k, a, p):
    if k == 0:
        return None

    k_binary = bin(k)[2:]
    Q = None

    for i in range(len(k_binary) - 1, -1, -1):
        Q = double_point(Q, a, p)
        if k_binary[len(k_binary) - 1-i] == '1':
            Q = add_points(Q, P, a, p)

    return Q


def jsf(k1, k2):
    k = {}
    k[1], k[2] = k1, k2
    l = 0
    d = {}
    d[1], d[2] = 0, 0
    K = {1:[], 2:[]}
    l_ = {}
    while (k[1] + d[1] > 0) or (k[2] + d[2] > 0):
        l_[1] = d[1] + k[1]
        l_[2] = d[2] + k[2]
        
        for i in range(1, 3):
            if mod(l_[i], 2) == 0:
                u = 0
            else:
                u = mod(l_[i], 4)
                if (mod(l_[i], 8) == 3 or mod(l_[i], 8) == -3) and mod(l_[3-i], 4) == 2:
                    u = -u
                
            if u == 3:
                K[i].append(-1)
            else:
                K[i].append(u)
        
        for i in range(1, 3):
            if 2*d[i] == 1 + K[i][l]:
                d[i] = 1 - d[i]
            k[i] = int(k[i]/2)
            
        l += 1
    
    return K


def precalculated(G, Q):
    list_points = {}
    
    list_points[5] = G + Q
    list_points[-5] = -G - Q
    list_points[2] = G
    list_points[3] = Q
    list_points[1] = -G + Q
    list_points[-1] = G - Q
    list_points[-2] = -G
    list_points[-3] = -Q
    list_points[0] = 0
    
    return list_points


def JSF(k1, k2, G, Q):
    start_time = time.time()
    list_points = precalculated(G, Q)
    print("pred =", time.time() - start_time)
    start_time = time.time()
    K = jsf(k1, k2)
    d = len(K[1])
    R = E(0, 1, 0)
    
    for i in range(d - 1, -1, -1):
        R = 2*R
        #R = R + int(K[1][i])*G + int(K[2][i])*Q
        R = R + list_points[int(2*K[1][i]) + int(3*K[2][i])]
    
    return R, time.time() - start_time


def test(G, Q):
    lens = [32, 64, 128, 256, 512, 1024, 2048, 4096]

    for i in lens:
        k1 = random.randint(2^i, 2^(i + 1))
        k2 = random.randint(2^i, 2^(i + 1))
        P, times = JSF(k1, k2, G, Q)
        
        start_time = time.time()
        P1 = binary_multiplication(G, k1, A, p) 
        P2 = binary_multiplication(Q, k2, A, p) 
        P = add_points(P1, P2, A, p)
        print("Sage =", time.time() - start_time)
        print("JSF time =", times)
        print()
      
    
p = 115792089237316193816632940749697632408932724669461758699102671622853342616127
A = 10001090682862736796505780343107022911835438229291751861888571869223050424594
B = 45264756867680555803214833811970559410867867042681754140959938453766481155105
Xg = 34131522064967883044818697086599304935071739419701303653954060188761196377435
Yg = 22161743160862324650134218095423264926581638655388631138889754360346138331114
Xq = 36165359026576437742151345460371859975651866220385050258939633540903065959717
Yq = 36943478257105992475915633328919578144889502533328778740461803868682833354169
k1 = 440044004412345678900987654321
k2 = 44^16

E = EllipticCurve(GF(p), [A, B])
G = E([Xg, Yg])
Q = E([Xq, Yq])

#print(E)
#print()
#print("JSF =", JSF(k1, k2, G, Q)[0])
#print("Sage =", k1*G + k2*Q)

test(G, Q)

pred = 0.00047087669372558594
Sage = 0.007170677185058594
JSF time = 0.007559776306152344

pred = 0.00032806396484375
Sage = 0.013678550720214844
JSF time = 0.009328603744506836

pred = 0.00034165382385253906
Sage = 0.028820514678955078
JSF time = 0.020031213760375977

pred = 0.0003311634063720703
Sage = 0.058868408203125
JSF time = 0.04083395004272461

pred = 0.00033974647521972656
Sage = 0.11354517936706543
JSF time = 0.0783836841583252

pred = 0.0005018711090087891
Sage = 0.22864127159118652
JSF time = 0.15839028358459473

pred = 0.0003571510314941406
Sage = 0.472689151763916
JSF time = 0.32462477684020996

pred = 0.0003390312194824219
Sage = 0.8953580856323242
JSF time = 0.6511313915252686

